In [1]:
import numpy as np
import shapley 
import RDG_interventional

f1: xs[0] + np.exp(xs[1] / (2 + xs[4])) + 2*(xs[2] * xs[3]) + np.sin(2*(xs[5] + xs[6]))


f2: 2*(np.sign(xs[0])) + np.sign(xs[1]*xs[2]*xs[3]) + np.sign(xs[4]*xs[5]*xs[6]) 

In [75]:
def f(xs):
    
    if type(xs).__module__ == np.__name__:
        #print(xs.shape)
        xs = xs.reshape(xs.shape[0],xs.shape[1])
        #print(xs.shape)
        #2*(np.sign(xs[0])) + np.sign(xs[1]*xs[2]*xs[3]) + np.sign(xs[4]*xs[5]*xs[6])
        #2*(np.sign(xs[0])) + np.sign(xs[1]*xs[2]*xs[3]) + np.sign(xs[4]*xs[5]*xs[6])
        #i = xs[0] + np.exp(xs[1] / (2 + xs[4])) + 2*(xs[2] * xs[3]) + np.sin(2*(xs[5] + xs[6]))
        #i = (np.sign(xs[0])) + (np.sign(xs[1]*xs[2]*xs[3])) + np.sign(xs[4]*xs[5]*xs[6])
        i = 2*(xs[:,0] * xs[:,2] * xs[:,3]) + 4*(xs[:,4] * xs[:,5]) - 3*(xs[:,1])**2 - (xs[:,6])
        
        return(i)
        
        #return ((xs[:,0] and xs[:,4]) or (xs[:,1] or xs[:,2])) + xs[:,3]
        #return xs[:,0] + np.exp(xs[:,1] / (2 + xs[:,4])) + 2*(xs[:,2] * xs[:,3]) + np.sin(xs[:,5] + xs[:,6])
    else:
        #return ((xs[0] and xs[4]) or (xs[1] or xs[2])) + xs[3]
        #return (xs[0] and (xs[1] or xs[2])) or ((xs[3] and not xs[4]) or xs[5])
        #return xs[0] + np.exp(xs[1] / (2 + xs[4])) + 2*(xs[2] * xs[3]) + np.sin(xs[5] + xs[6])
        #i =  xs[0] + np.exp(xs[1] / (2 + xs[4])) + 2*(xs[2] * xs[3]) + np.sin(2*(xs[5] + xs[6]))
    
        #i = (np.sign(xs[0])) + (np.sign(xs[1]*xs[2]*xs[3])) + np.sign(xs[4]*xs[5]*xs[6])
        i = 2*(xs[0] * xs[2] * xs[3]) + 4*(xs[4] * xs[5]) - 3*(xs[1])**2 - (xs[6])
        
        return(i)

In [76]:
#x = [1 for _ in range(0,7)]
x = [np.random.uniform(-1,1) for _ in range(0,7)]

z = [np.random.uniform(-1,1) for _ in range(0,7)]


In [77]:
ss_atts,seps,nonseps = RDG_interventional.RDG(f,list(x),list(z),[-1 for _ in range(0,len(x))],[1 for _ in range(0,len((x)))],0.1)
print(ss_atts)

{(1,): array([-2.08126763]), (6,): array([1.52647463]), (0, 2, 3): array([-0.0444675]), (4, 5): array([0.75085969])}


In [78]:
s_atts = shapley.shapley_int(f,np.asarray(x),np.asarray(z))
s_atts

{0: array([0.05039687]),
 1: array([-2.08126763]),
 2: array([0.00939059]),
 3: array([-0.10425497]),
 4: array([-0.10648528]),
 5: array([0.85734497]),
 6: array([1.52647463])}

In [79]:
import sklearn
import shap
from sklearn.model_selection import train_test_split
import random

datas = []
for i in range(100):
    data = []
    for i in range(0,3):
        data.append(random.uniform(-1,1))
    datas.append(data)
datas = np.asarray(datas).reshape(100,3)
explainer = shap.KernelExplainer(f, datas)
shap_values = explainer.shap_values(np.asarray(x).reshape(1,len(x)), nsamples=100)

ks_atts = {}
for i in range(0,len(x)):
    ks_atts[i] = shap_values[0][i]
ks_atts

Provided model function fails when applied to the provided data set.


IndexError: index 3 is out of bounds for axis 1 with size 3

In [80]:
from numpy import trapz
from sklearn import metrics

def faithfulness(f, atts, x, z):
    if f(list(z)) < f(list(x)):
        sorted_indexes = sorted(atts, key=atts.get, reverse=True)
    else:
        sorted_indexes = sorted(atts, key=atts.get)
    print(sorted_indexes)
    predictions = []
    predictions.append(f(list(x)))
    new_x = x.copy()
    for j in sorted_indexes:
        j = np.asarray(j)
        new_x[j] = z[j]
        predictions.append(f(list(new_x)))
    predictions.append(f(list(z)))
    print(predictions)
    #fpr, tpr, thresholds = metrics.roc_curve(np.arange(len(sorted_indexes)), predictions, pos_label=2)
    #return metrics.auc(fpr, tpr)

In [81]:
auc = faithfulness(f,ss_atts,np.asarray(x),np.asarray(z))
auc

[(6,), (4, 5), (0, 2, 3), (1,)]
[-2.3985033384645664, -3.9249779726897462, -4.675837666750379, -4.631370166333636, -2.5501025357683322, -2.5501025357683322]


In [82]:
auc = faithfulness(f,s_atts,np.asarray(x),np.asarray(z))
auc

[6, 5, 0, 2, 3, 4, 1]
[-2.3985033384645664, -3.9249779726897462, -2.6282726465082007, -3.0232948183834307, -2.9735145962765492, -2.583805146091458, -4.631370166333636, -2.5501025357683322, -2.5501025357683322]


In [83]:
def robustness(f,x,z,att_map):
    corrupted_x = x * 0.99
    if att_map == 'ss':
        atts,seps,nonseps = RDG_interventional.RDG(f,list(x),list(z))
    elif att_map == 's':
        atts = shapley.shapley_int(f,np.asarray(list(x)),list(z))
    if att_map == 'ss':
        corrupted_atts,seps,nonseps = RDG_interventional.RDG(f,list(corrupted_x),list(z))
    elif att_map == 's':
        corrupted_atts = shapley.shapley_int(f,np.asarray(list(corrupted_x)),list(z))
   
    sorted_indexes = sorted(atts, key=atts.get, reverse=True)
    sorted_corrupted_indexes = sorted(atts, key=atts.get, reverse=True)

    print(sorted_indexes)
    print(sorted_corrupted_indexes)

In [84]:
#robustness(f,x,z,'ss')

In [85]:
#robustness(f,x,z,'s')

In [86]:
def deletion():
    overall_s_error = []
    overall_ss_error = []
    overall_ks_error = []
    for i in range(0,1):
        x = [np.random.uniform(-1,1) for _ in range(0,7)]
        z = [0 for _ in range(0,7)]
        ss_atts,seps,nonseps = RDG_interventional.RDG(f,list(x),list(z),[-1 for _ in range(0,len(x))],[1 for _ in range(0,len((x)))],0)
        s_atts = shapley.shapley_int(f,np.asarray(x),np.asarray(z))
        
        for att in s_atts.keys():
            s_atts[att] = np.abs(s_atts[att])
        for att in ss_atts.keys():
            ss_atts[att] = np.abs(ss_atts[att])
        
        
        ss_sorted_indexes = sorted(ss_atts, key=ss_atts.get, reverse=True)
        s_sorted_indexes = sorted(s_atts, key=s_atts.get, reverse=True)

        
        
        z = np.asarray(z)
        s_predictions = []
        s_predictions.append(f(list(x)))
        new_x = np.asarray(x.copy())
        for j in s_sorted_indexes:
            new_x = np.asarray(x.copy())
            print(j)
            j = np.asarray(j)
            new_x[j] = z[j]
            s_predictions.append(f(list(new_x)))
        s_predictions.append(f(list(z)))
        
        ss_predictions = []
        ss_predictions.append(f(list(x)))
        new_x = np.asarray(x.copy())
        for j in ss_sorted_indexes:
            new_x = np.asarray(x.copy())
            j = np.asarray(j)
            print(j)
            new_x[j] = z[j]
            ss_predictions.append(f(list(new_x)))
        ss_predictions.append(f(list(z)))
        
        print(s_predictions)
        print(ss_predictions)
        
        
        
        
    return(overall_s_error, overall_ss_error, overall_ks_error)



In [107]:
def MSE_f1():
    overall_s_error = []
    overall_ss_error = []
    overall_ks_error = []
    for i in range(0,10):
        x = [np.random.uniform(-1,1) for _ in range(0,7)]
        z = [0 for _ in range(0,7)]
        ss_atts,seps,nonseps = RDG_interventional.RDG(f,list(x),list(z),[-1 for _ in range(0,len(x))],[1 for _ in range(0,len((x)))],0.2)
        s_atts = shapley.shapley_int(f,np.asarray(x),np.asarray(z))
        
        print(ss_atts)
        print(ground_truth)
        
        datas = []
        for i in range(100):
            data = []
            for i in range(0,len(x)):
                data.append(random.uniform(-1,1))
            datas.append(data)
        datas = np.asarray(datas).reshape(100,len(x))
        #explainer = shap.KernelExplainer(f, datas)
        #shap_values = explainer.shap_values(np.asarray(x).reshape(1,len(x)), nsamples=1000)

        """ks_atts = {}
        for i in range(0,len(x)):
            ks_atts[i] = shap_values[0][i]"""
            
        
      
        ground_truth = {}
        ground_truth[0] = x[0] - z[0]
        ground_truth[1] = (np.exp(x[1] / (2 + x[4])))  - (np.exp(z[1] / (2 + z[4]))) 
        ground_truth[2] = (2*(x[2] * x[3])) - ((z[2] * z[3]))
        ground_truth[3] =  (2*(x[2] * x[3])) - ((z[2] * z[3]))
        ground_truth[4] = (np.exp(x[1] / (2 + x[4])))  - (np.exp(z[1] / (2 + z[4]))) 
        ground_truth[5] =  (np.sin(2*(x[5] + x[6]))) - (np.sin(2*(z[5] + z[6])))
        ground_truth[6] =  (np.sin(2*(x[5] + x[6]))) - (np.sin(2*(z[5] + z[6])))


        ss_error = []
        for i in range(0,len(x)):
            u_index = 0
            for u in ss_atts.keys():
                if i in u:
                    ss_error.append(np.abs(ground_truth[i] - list(ss_atts.values())[u_index]))
                u_index +=1
        #print(np.mean(np.asarray(ss_error)))
        s_error = []
        for i in range(0,len(x)):
            u_index = 0
            for u in s_atts.keys():
                if i == u:
                    s_error.append(np.abs(ground_truth[i] - list(s_atts.values())[u_index]))
                u_index +=1
                
        ks_error = []
        for i in range(0,len(x)):
            u_index = 0
            for u in ks_atts.keys():
                if i == u:
                    ks_error.append(np.abs(ground_truth[i] - list(ks_atts.values())[u_index]))
                u_index +=1
                
        #print(np.mean(np.asarray(s_error)))
        overall_s_error.append(s_error)
        overall_ss_error.append(ss_error)
        overall_ks_error.append(ks_error)
    return(overall_s_error, overall_ss_error, overall_ks_error)



In [108]:
def MSE():
    overall_s_error = []
    overall_ss_error = []
    overall_ks_error = []
    for i in range(0,1):
        x = [np.random.uniform(-1,1) for _ in range(0,7)]
        z = [np.random.uniform(-1,1) for _ in range(0,7)]
        ss_atts,seps,nonseps = RDG_interventional.RDG(f,list(x),list(z),[-1 for _ in range(0,len(x))],[1 for _ in range(0,len((x)))],0.1)
        s_atts = shapley.shapley_int(f,np.asarray(x),np.asarray(z))
        
        
        
        """datas = []
        for i in range(100):
            data = []
            for i in range(0,len(x)):
                data.append(random.uniform(-1,1))
            datas.append(data)
        datas = np.asarray(datas).reshape(100,len(x))
        explainer = shap.KernelExplainer(f, datas)
        shap_values = explainer.shap_values(np.asarray(x).reshape(1,len(x)), nsamples=1000)

        ks_atts = {}
        for i in range(0,len(x)):
            ks_atts[i] = shap_values[0][i]"""
            
        
      
        ground_truth = {}
        ground_truth[0] = 2*(x[0] * x[2] * x[3]) - 2*(z[0] * z[2] * z[3]) 
        ground_truth[1] =  (- 3*(x[1])**2) - ((- 3*(z[1])**2))
        ground_truth[2] = 2*(x[0] * x[2] * x[3]) - 2*(z[0] * z[2] * z[3])
        ground_truth[3] =  2*(x[0] * x[2] * x[3]) - 2*(z[0] * z[2] * z[3])
        ground_truth[4] = 4*(x[4] * x[5]) - 4*(z[4] * z[5])
        ground_truth[5] =  4*(x[4] * x[5]) - 4*(z[4] * z[5])
        ground_truth[6] =  (- (x[6])) - (- (z[6]))
        
        print(ss_atts)
        print(ground_truth)

        ss_error = []
        for i in range(0,len(x)):
            u_index = 0
            for u in ss_atts.keys():
                if i in u:
                    ss_error.append(np.abs(ground_truth[i] - list(ss_atts.values())[u_index]))
                u_index +=1
        #print(np.mean(np.asarray(ss_error)))
        s_error = []
        for i in range(0,len(x)):
            u_index = 0
            for u in s_atts.keys():
                if i == u:
                    s_error.append(np.abs(ground_truth[i] - list(s_atts.values())[u_index]))
                u_index +=1
                
        """ks_error = []
        for i in range(0,len(x)):
            u_index = 0
            for u in ks_atts.keys():
                if i == u:
                    ks_error.append(np.abs(ground_truth[i] - list(ks_atts.values())[u_index]))
                u_index +=1"""
                
        #print(np.mean(np.asarray(s_error)))
        overall_s_error.append(s_error)
        overall_ss_error.append(ss_error)
        #overall_ks_error.append(ks_error)
    return(overall_s_error, overall_ss_error)



In [113]:
overall_s_error, overall_ss_error = MSE()

{(1,): array([0.91714314]), (6,): array([-0.47343521]), (0, 2, 3): array([-0.42987555]), (4, 5): array([3.51581013])}
{0: -0.4298755518804329, 1: 0.9171431363179482, 2: -0.4298755518804329, 3: -0.4298755518804329, 4: 3.515810127783822, 5: 3.515810127783822, 6: -0.4734352069899668}
